In [1]:
from preproc import *
from tqdm import tqdm
import spacy
import ujson as json
from collections import Counter
import numpy as np
from codecs import open
from config import config
import sys
from absl import app
from config import config, device
from preproc import preproc
from absl import app
from math import log2
import os
import numpy as np
import ujson as json
import re
from collections import Counter
import string
from tqdm import tqdm
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.cuda
from torch.utils.data import Dataset

In [ ]:
class SQuADDataset(Dataset):
    def __init__(self, npz_file, num_steps, batch_size):
        super().__init__()
        data = np.load(npz_file)
        self.context_idxs = torch.from_numpy(data["context_idxs"]).long()
        self.context_char_idxs = torch.from_numpy(data["context_char_idxs"]).long()
        self.ques_idxs = torch.from_numpy(data["ques_idxs"]).long()
        self.ques_char_idxs = torch.from_numpy(data["ques_char_idxs"]).long()
        self.y1s = torch.from_numpy(data["y1s"]).long()
        self.y2s = torch.from_numpy(data["y2s"]).long()
        self.ids = torch.from_numpy(data["ids"]).long()
        num = len(self.ids)
        self.batch_size = batch_size
        self.num_steps = num_steps if num_steps >= 0 else num // batch_size
        num_items = num_steps * batch_size
        idxs = list(range(num))
        self.idx_map = []
        i, j = 0, num

        while j <= num_items:
            random.shuffle(idxs)
            self.idx_map += idxs.copy()
            i = j
            j += num
        random.shuffle(idxs)
        self.idx_map += idxs[:num_items - i]

    def __len__(self):
        return self.num_steps

    def __getitem__(self, item):
        idxs = torch.LongTensor(self.idx_map[item:item + self.batch_size])
        res = (self.context_idxs[idxs],
               self.context_char_idxs[idxs],
               self.ques_idxs[idxs],
               self.ques_char_idxs[idxs],
               self.y1s[idxs],
               self.y2s[idxs], self.ids[idxs])
        return res



In [ ]:
word_counter, char_counter = Counter(), Counter()
train_examples, train_eval = process_file('data/dev-v1.1.json', "train", word_counter, char_counter)

In [ ]:
# config(sys.argv)
word_emb_file = config.glove_word_file  
char_emb_file = config.glove_char_file if config.pretrained_char else None
char_emb_size = config.glove_char_size if config.pretrained_char else None
char_emb_dim = config.glove_dim if config.pretrained_char else config.char_dim

In [2]:
%run -m main -- --mode data

# def main(_):
#     from config import config
    
# if __name__ == '__main__':
#     app.run(main)

  0%|          | 0/48 [00:00<?, ?it/s]

Generating train examples...


100%|██████████| 48/48 [00:07<00:00,  6.36it/s]
1039it [00:00, 10381.37it/s]

10570 questions in total
Generating word embedding...


2196018it [03:14, 11277.95it/s]
383it [00:00, 3826.87it/s]

...

4850
4851
4852
4853
5444
5445
5446
5447
5448


7895it [00:02, 3476.05it/s]

7331
7332
7333
7334
7335
7336
7337
7338
7339
7340


10570it [00:03, 3441.47it/s]


Built 10482 / 10570 instances of features in total
Saving word embedding...
Saving char embedding...
Saving train eval...
Saving word dictionary...
Saving char dictionary...
Saving dev meta...


In [ ]:
os.getcwd()

In [ ]:
import json
with open('./data/train_eval.json') as f:
    data = json.load(f)

In [ ]:
data

In [ ]:
train_dataset = SQuADDataset(config.train_record_file, config.num_steps, config.batch_size)
dev_dataset = SQuADDataset(config.dev_record_file, config.test_num_batches, config.batch_size)
data = np.load('./data/train.npz')

In [ ]:
data["context_idxs"]

In [ ]:
len(train_dataset.y1s[train_dataset.y2s == 0])

In [ ]:
data["y1s"]

In [ ]:
build_features(config, train_examples, 'train', config.train_record_file,\
               word2idx_dict, char2idx_dict )